In [ ]:
import matplotlib.pyplot as plt
import msprime
import numpy as np
from pathlib import Path
import random
import tskit
import os
import pandas as pd
import seaborn as sns

In [ ]:
def get_population_time(time_rate:float=0.06, tmax:int = 130_000,
                        num_time_windows:int = 21
                       ) -> np.array :
    population_time = np.repeat([(np.exp(np.log(1 + time_rate * tmax) * i /
                              (num_time_windows - 1)) - 1) / time_rate for i in
                              range(num_time_windows)], 1, axis=0)
    population_time[0] = 1
    return population_time



def get_updated_population_timey(steps = 18):
    x = np.log(get_population_time(time_rate=0.1, num_time_windows=steps, tmax=10_000_000).tolist())
    xnew = np.linspace(x[0], x[-1], num=10000, endpoint=True)
    x_sample = xnew[np.linspace(10, 9999, 60).astype(int)]
    population_time = np.exp(x_sample)
    return population_time

population_time = get_updated_population_timey()

In [ ]:
os.system("mkdir vcf")
os.system("mkdir trees")
os.system("mkdir tcoalmsp")

0

In [ ]:
vcfdir='./vcf/'
tcdir='./tcoalmsp/'
tsdir='./trees/'
mkdir = "./sites/"

nrep=1

In [ ]:
population_time = get_updated_population_timey()

In [ ]:
def kingman_sawtooth(Ne=10**4, L=10_000_000, sample_size = 10, r = 1e-8, m=1e-7, ploidy = 1):   
    
    
    pref = "r" + str(r) + "m" + str(m) + "_"
    alpha = 2.0
    demography=msprime.Demography()
    demography.add_population(initial_size=(Ne))
    demography.add_population_parameters_change(time=20, population=None, growth_rate=6437.7516497364/(4*10**4))
    demography.add_population_parameters_change(time=30, growth_rate=-378.691273513906/(4*10**4))
    demography.add_population_parameters_change(time=200, growth_rate=-643.77516497364/(4*10**4))
    demography.add_population_parameters_change(time=300, growth_rate=37.8691273513906/(4*10**4))
    demography.add_population_parameters_change(time=2000, growth_rate=64.377516497364/(4*10**4))
    demography.add_population_parameters_change(time=3000, growth_rate=-3.78691273513906/(4*10**4))
    demography.add_population_parameters_change(time=20000, growth_rate=-6.4377516497364/(4*10**4))
    demography.add_population_parameters_change(time=30000, growth_rate=0.378691273513906/(4*10**4))
    demography.add_population_parameters_change(time=200000, growth_rate=0.64377516497364/(4*10**4))
    demography.add_population_parameters_change(time=300000, growth_rate=-0.0378691273513906/(4*10**4))
    demography.add_population_parameters_change(time=2000000, growth_rate=-0.064377516497364/(4*10**4))
    demography.add_population_parameters_change(time=3000000, growth_rate=0.00378691273513906/(4*10**4))
    demography.add_population_parameters_change(time=20000000, growth_rate=0, initial_size=Ne)
    db = msprime.DemographyDebugger(demography=demography)
    tree_sequences = []
    
    
    n = sample_size
    
    for i in range(10):
        
        tree_sequence = msprime.sim_ancestry(samples=sample_size, recombination_rate=r,sequence_length=L, demography=demography,ploidy=1,random_seed=((alpha*i+1)**2))
        tree_sequence = msprime.mutate(tree_sequence, rate=m)
        
        vcffh = open(vcfdir+'kigman_sim_'+pref+str(i)+'.vcf', 'w')
        tree_sequence.write_vcf(vcffh, position_transform='legacy', individual_names=['spl'+str(s) for s in range(int(n))])
        vcffh.close()
        
        tsfile = tsdir+'sim_'+pref+str(i)+'.trees'
        tree_sequence.dump(tsfile)
        for s1 in range(0, n-1):
            for s2 in range(s1+1, n):
                coaltimefh = open(tcdir+'/kingman_sim_'+pref + str(i)+'_spls'+str(s1)+'-'+str(s2)+'.tc', 'w')
                for tree in tree_sequence.trees(): 
                    treeInterval = tree.get_interval() # length of sequence that this tree spans
                    coalescence_time = tree.tmrca(s1,s2)
                    print(treeInterval[0], treeInterval[1], coalescence_time, sep = "\t", file = coaltimefh)
                coaltimefh.close()
        
        
        tree_sequences.append(tree_sequence)
    demography = db.population_size_trajectory(population_time).flatten()
    return list(tree_sequences), demography 

In [ ]:
def beta_sawtooth(alpha, Ne=10**6, L=10_000_000, sample_size = 10, r = 1e-8, m=1e-7, ploidy = 1):   
    
    pref = "r" + str(r) + "m" + str(m) + "_"
    demography=msprime.Demography()
    demography.add_population(initial_size=(Ne))
    demography.add_population_parameters_change(time=20, population=None, growth_rate=6437.7516497364/(4*10**4))
    demography.add_population_parameters_change(time=30, growth_rate=-378.691273513906/(4*10**4))
    demography.add_population_parameters_change(time=200, growth_rate=-643.77516497364/(4*10**4))
    demography.add_population_parameters_change(time=300, growth_rate=37.8691273513906/(4*10**4))
    demography.add_population_parameters_change(time=2000, growth_rate=64.377516497364/(4*10**4))
    demography.add_population_parameters_change(time=3000, growth_rate=-3.78691273513906/(4*10**4))
    demography.add_population_parameters_change(time=20000, growth_rate=-6.4377516497364/(4*10**4))
    demography.add_population_parameters_change(time=30000, growth_rate=0.378691273513906/(4*10**4))
    demography.add_population_parameters_change(time=200000, growth_rate=0.64377516497364/(4*10**4))
    demography.add_population_parameters_change(time=300000, growth_rate=-0.0378691273513906/(4*10**4))
    demography.add_population_parameters_change(time=2000000, growth_rate=-0.064377516497364/(4*10**4))
    demography.add_population_parameters_change(time=3000000, growth_rate=0.00378691273513906/(4*10**4))
    demography.add_population_parameters_change(time=20000000, growth_rate=0, initial_size=Ne)
    db = msprime.DemographyDebugger(demography=demography)
    tree_sequences = []
    
    n = sample_size
    for i in range(10):
        
        tree_sequence = msprime.sim_ancestry(samples=sample_size, recombination_rate=r,sequence_length=L, demography=demography,ploidy=1,random_seed=((alpha*i+1)**2), model=msprime.BetaCoalescent(alpha=alpha))
        tree_sequence = msprime.mutate(tree_sequence, rate=m)
        
        vcffh = open(vcfdir+'beta_sim_' + str(alpha) + '_' + pref + str(i)+'.vcf', 'w')
        
        tree_sequence.write_vcf(vcffh, position_transform='legacy', individual_names=['spl'+str(s) for s in range(int(n))])
        vcffh.close()
        
        tsfile = tsdir+'beta_sim_' + str(alpha) + "_" + pref + str(i)+'.trees'
        tree_sequence.dump(tsfile)
        for s1 in range(0, n-1):
            for s2 in range(s1+1, n):
                coaltimefh = open(tcdir+'/beta_' + str(alpha) + '_sim_'+ pref + str(i)+'_spls'+str(s1)+'-'+str(s2)+'.tc', 'w')
                for tree in tree_sequence.trees(): 
                    treeInterval = tree.get_interval() # length of sequence that this tree spans
                    coalescence_time = tree.tmrca(s1,s2)
                    print(treeInterval[0], treeInterval[1], coalescence_time, sep = "\t", file = coaltimefh)
                coaltimefh.close()
        
        tree_sequences.append(tree_sequence)
    demography = db.population_size_trajectory(population_time).flatten()
    return list(tree_sequences), demography 

In [ ]:
#tree_sequences, demography = kingman_sawtooth()

tree_sequences_b19, demography = beta_sawtooth(1.9, r = 1e-9, m=1e-8, L=10_000_000)
tree_sequences_b17, demography = beta_sawtooth(1.7, r = 1e-8, m=1e-7, L=10_000_000)
tree_sequences_b15, demography = beta_sawtooth(1.5, r = 1e-8, m=1e-7, L=10_000_000)
tree_sequences_b13, demography = beta_sawtooth(1.3, r = 1e-7, m=1e-6, L=10_000_000)

In [ ]:
#tree_sequences, demography = kingman_sawtooth()

tree_sequences_b19, demography = beta_sawtooth(1.9, r = 1e-8, m=1e-8, L=10_000_000)
tree_sequences_b17, demography = beta_sawtooth(1.7, r = 1e-8, m=1e-8, L=10_000_000)
tree_sequences_b15, demography = beta_sawtooth(1.5, r = 1e-7, m=1e-7, L=10_000_000)
tree_sequences_b13, demography = beta_sawtooth(1.3, r = 1e-6, m=1e-6, L=10_000_000)

In [ ]:
tree_sequences_b19[0].num_trees, tree_sequences_b17[0].num_trees, tree_sequences_b15[0].num_trees, tree_sequences_b13[0].num_trees

(1079, 3685, 917, 652)

In [ ]:
r = 1e-8
m = 1e-7

pref = "r" + str(r) + "m" + str(m) + "_"

In [ ]:
os.system("mkdir sites")

0

In [ ]:
alpha_dict = {1.9: [1e-8, 1e-8], 1.7:[1e-8, 1e-8], 1.5:[1e-7, 1e-7], 1.3:[1e-6, 1e-6]}

In [ ]:
alpha = 1.9

for alpha in [1.9, 1.7, 1.5, 1.3]:
    
    seqlen = str(10_000_000)
    r, m = alpha_dict[alpha]
    
    for i in range(10):

        pref = "r" + str(r) + "m" + str(m) + "_"
        vcffile = "./vcf/beta_sim_" + str(alpha) + "_" + str(pref) + str(i) +".vcf"
        nspl = 5
        outdir = "./sites"

        infh = open(vcffile, 'r')
        pref = vcffile.split('/')[-1].split('.vcf')[0]
        outfile = pref+'.sites'

        print(outfile)

        outfh = open(outdir+'/'+outfile, 'w')
        outfh.write('\t'.join(['NAMES']+['spl'+str(i)+'_1'+'\t'+'spl'+str(i)+'_2' for i in range(nspl)])+'\n')
        outfh.write('REGION\t1\t1\t'+seqlen+'\n')

        allpos=[]
        for line in infh:
            if line.startswith('#'):
                continue
            fields = line.split()
            pos = int(fields[1])
            while True:
                if pos in allpos:
                    pos = pos+1
                else:
                    break

            allpos.append(pos)
            alleles_01 = []
            [alleles_01.extend(x.split('|')) for x in fields[9:]]
            alleles_tmp = [x.replace('0', 'A') for x in alleles_01]
            #two lines below added for JC finite sites simulation
            alleles_tmp = [x.replace('2', 'G') for x in alleles_tmp]
            alleles_tmp = [x.replace('3', 'T') for x in alleles_tmp]
            alleles_nuc = [x.replace('1', 'C') for x in alleles_tmp]
            outfh.write(str(pos)+'\t'+''.join(alleles_nuc)+'\n')

        infh.close()
        outfh.close()

beta_sim_1.9_r1e-08m1e-08_0.sites
beta_sim_1.9_r1e-08m1e-08_1.sites
beta_sim_1.9_r1e-08m1e-08_2.sites
beta_sim_1.9_r1e-08m1e-08_3.sites
beta_sim_1.9_r1e-08m1e-08_4.sites
beta_sim_1.9_r1e-08m1e-08_5.sites
beta_sim_1.9_r1e-08m1e-08_6.sites
beta_sim_1.9_r1e-08m1e-08_7.sites
beta_sim_1.9_r1e-08m1e-08_8.sites
beta_sim_1.9_r1e-08m1e-08_9.sites
beta_sim_1.7_r1e-08m1e-08_0.sites
beta_sim_1.7_r1e-08m1e-08_1.sites
beta_sim_1.7_r1e-08m1e-08_2.sites
beta_sim_1.7_r1e-08m1e-08_3.sites
beta_sim_1.7_r1e-08m1e-08_4.sites
beta_sim_1.7_r1e-08m1e-08_5.sites
beta_sim_1.7_r1e-08m1e-08_6.sites
beta_sim_1.7_r1e-08m1e-08_7.sites
beta_sim_1.7_r1e-08m1e-08_8.sites
beta_sim_1.7_r1e-08m1e-08_9.sites
beta_sim_1.5_r1e-07m1e-07_0.sites
beta_sim_1.5_r1e-07m1e-07_1.sites
beta_sim_1.5_r1e-07m1e-07_2.sites
beta_sim_1.5_r1e-07m1e-07_3.sites
beta_sim_1.5_r1e-07m1e-07_4.sites
beta_sim_1.5_r1e-07m1e-07_5.sites
beta_sim_1.5_r1e-07m1e-07_6.sites
beta_sim_1.5_r1e-07m1e-07_7.sites
beta_sim_1.5_r1e-07m1e-07_8.sites
beta_sim_1.5_r

In [ ]:
import os
import gzip
import newick
import networkx as nx
from io import StringIO
from Bio import Phylo
import pickle
import tskit
import pandas as pd
import re

In [ ]:
nrep = -1
for rep in [0, 1, 2, 3, 4, 5, 6 ,7, 8 ,9]: #3, 4, 5, 6, 7, 8, 9
    try:
        for mcmc_step in [950, 1000]: #960, 980, 1000
            
            files = [file for file in os.listdir("weaved_args") if "beta_1.9_arg-sample_r1e-08m1e-08_1-10000000_" + str(rep) in file and ".smc.gz" in file]
            files = [file for file in files if int(file.split(".")[-3]) == mcmc_step]


            file = files[0]
            print(file)
            with gzip.open("./weaved_args/" + file,'rb')as f:
                newick_trees = [line for line in f if b"TREE" in line]  

            clean_newick_trees = []
            for newick_tree in newick_trees:
                
                newick_tree = re.split(b'\t+', newick_trees[0])[-1][:-1]
                newick_tree = newick_tree.decode("utf-8") 
                newick_tree = re.sub("([\[]).*?([\]])", "\g<1>\g<2>", newick_tree)
                newick_tree = re.sub("[\[].*?[\]]", "", newick_tree)
                clean_newick_trees.append(newick_tree)

            for i, n in enumerate(clean_newick_trees):

                tree = Phylo.read(StringIO(n), "newick")
                nx_tree = Phylo.to_networkx(tree)
                graph = nx.Graph(nx_tree)
                
                with open('./graphs19/beta_graph_' + str(mcmc_step) + "_" + str(i) + "_rep" + str(rep) +'.pickle', 'wb') as handle:
                    pickle.dump(graph, handle)
    except Exception as e:
        print(e, rep, mcmc_step)

In [ ]:
nrep = -1
for rep in [0, 1, 2, 3, 4, 5, 6 ,7, 8 ,9]: #3, 4, 5, 6, 7, 8, 9
    try:
        for mcmc_step in [950, 1000]: #960, 980, 1000
            
            files = [file for file in os.listdir("weaved_args") if "beta_1.7_arg-sample_r1e-08m1e-08_1-10000000_" + str(rep) in file and ".smc.gz" in file]
            files = [file for file in files if int(file.split(".")[-3]) == mcmc_step]

            file = files[0]
            print(file)
            with gzip.open("./weaved_args/" + file,'rb')as f:
                newick_trees = [line for line in f if b"TREE" in line]  

            clean_newick_trees = []
            for newick_tree in newick_trees:
                
                newick_tree = re.split(b'\t+', newick_trees[0])[-1][:-1]
                newick_tree = newick_tree.decode("utf-8") 
                newick_tree = re.sub("([\[]).*?([\]])", "\g<1>\g<2>", newick_tree)
                newick_tree = re.sub("[\[].*?[\]]", "", newick_tree)
                clean_newick_trees.append(newick_tree)

            for i, n in enumerate(clean_newick_trees):

                tree = Phylo.read(StringIO(n), "newick")
                nx_tree = Phylo.to_networkx(tree)
                graph = nx.Graph(nx_tree)
                
                with open('./graphs17/beta_graph_' + str(mcmc_step) + "_" + str(i) + "_rep" + str(rep) +'.pickle', 'wb') as handle:
                    pickle.dump(graph, handle)
    except:
        print(rep, mcmc_step)

In [ ]:
nrep = -1
for rep in [0, 1, 2, 3, 4, 5, 6 ,7, 8 ,9]: #3, 4, 5, 6, 7, 8, 9
    try:
        for mcmc_step in [950, 1000]: #960, 980, 1000
            
            files = [file for file in os.listdir("weaved_args") if "beta_1.5_arg-sample_r1e-07m1e-07_1-10000000_" + str(rep) in file and ".smc.gz" in file]
            files = [file for file in files if int(file.split(".")[-3]) == mcmc_step]


            file = files[0]
            print(file)
            with gzip.open("./weaved_args/" + file,'rb')as f:
                newick_trees = [line for line in f if b"TREE" in line]  

            clean_newick_trees = []
            for newick_tree in newick_trees:
                
                newick_tree = re.split(b'\t+', newick_trees[0])[-1][:-1]
                newick_tree = newick_tree.decode("utf-8") 
                newick_tree = re.sub("([\[]).*?([\]])", "\g<1>\g<2>", newick_tree)
                newick_tree = re.sub("[\[].*?[\]]", "", newick_tree)
                clean_newick_trees.append(newick_tree)

            for i, n in enumerate(clean_newick_trees):

                tree = Phylo.read(StringIO(n), "newick")
                nx_tree = Phylo.to_networkx(tree)
                graph = nx.Graph(nx_tree)
                
                with open('./graphs15/beta_graph_' + str(mcmc_step) + "_" + str(i) + "_rep" + str(rep) +'.pickle', 'wb') as handle:
                    pickle.dump(graph, handle)
    except:
        print(rep, mcmc_step)

In [ ]:
nrep = -1
for rep in [0, 1, 2, 3, 4, 5, 6 ,7, 8 ,9]: #3, 4, 5, 6, 7, 8, 9
    try:
        for mcmc_step in [950, 1000]: #960, 980, 1000
            
            files = [file for file in os.listdir("weaved_args") if "beta_1.3_arg-sample_r1e-06m1e-06_1-10000000_" + str(rep) in file and ".smc.gz" in file]
            files = [file for file in files if int(file.split(".")[-3]) == mcmc_step]


            file = files[0]
            print(file)
            with gzip.open("./weaved_args/" + file,'rb')as f:
                newick_trees = [line for line in f if b"TREE" in line]  

            clean_newick_trees = []
            for newick_tree in newick_trees:
                
                newick_tree = re.split(b'\t+', newick_trees[0])[-1][:-1]
                newick_tree = newick_tree.decode("utf-8") 
                newick_tree = re.sub("([\[]).*?([\]])", "\g<1>\g<2>", newick_tree)
                newick_tree = re.sub("[\[].*?[\]]", "", newick_tree)
                clean_newick_trees.append(newick_tree)

            for i, n in enumerate(clean_newick_trees):

                tree = Phylo.read(StringIO(n), "newick")
                nx_tree = Phylo.to_networkx(tree)
                graph = nx.Graph(nx_tree)
                
                with open('./graphs13/beta_graph_' + str(mcmc_step) + "_" + str(i) + "_rep" + str(rep) +'.pickle', 'wb') as handle:
                    pickle.dump(graph, handle)
    except:
        print(rep, mcmc_step)